In [1]:
import os

In [2]:
%pwd

'/workspaces/project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2024-03-27 15:19:53.378656: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 15:19:53.411911: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 15:19:53.411943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 15:19:53.411977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-27 15:19:53.418413: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 15:19:53.419196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

In [17]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.base_model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
        destination_dir = path.parent.parent.parent / "model"
        if not destination_dir.exists():
            destination_dir.mkdir(parents=True)

        # Copy the file to the destination directory
        destination_file = destination_dir / "model.h5"
        shutil.copyfile(path, destination_file)


    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        for layer in self.base_model.layers:
            layer.trainable = False
        class_count = len(list(self.train_generator.class_indices.keys()))
        self.model = Sequential([
                self.base_model,
                BatchNormalization(axis=-1),
                Dense(256, activation='relu'),
                Dropout(0.3),
                Dense(class_count, activation='softmax')
            ])
        optimizer = tf.keras.optimizers.Adam()
        self.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        filepath = str(self.config.trained_model_path)
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=[checkpoint]
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [18]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-27 15:30:49,050: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-27 15:30:49,052: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-27 15:30:49,052: INFO: common: created directory at: artifacts]
[2024-03-27 15:30:49,053: INFO: common: created directory at: artifacts/training]
Found 182 images belonging to 4 classes.
Found 736 images belonging to 4 classes.
Epoch 1/100


2024-03-27 15:30:59.815442: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 115605504 exceeds 10% of free system memory.
2024-03-27 15:30:59.853243: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 115605504 exceeds 10% of free system memory.
2024-03-27 15:30:59.914537: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117679104 exceeds 10% of free system memory.


 1/46 [..............................] - ETA: 5:57 - loss: 1.3763 - accuracy: 0.2500

2024-03-27 15:31:00.573282: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 115605504 exceeds 10% of free system memory.
2024-03-27 15:31:00.604288: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 115605504 exceeds 10% of free system memory.


46/46 [==============================] - ETA: 0s - loss: 1.3715 - accuracy: 0.3207
Epoch 1: val_accuracy improved from -inf to 0.36932, saving model to artifacts/training/model.h5


/workspaces/project/env/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


46/46 [==============================] - 48s 881ms/step - loss: 1.3715 - accuracy: 0.3207 - val_loss: 1.3571 - val_accuracy: 0.3693
Epoch 2/100
46/46 [==============================] - ETA: 0s - loss: 1.3434 - accuracy: 0.3465
Epoch 2: val_accuracy did not improve from 0.36932
46/46 [==============================] - 34s 733ms/step - loss: 1.3434 - accuracy: 0.3465 - val_loss: 1.3496 - val_accuracy: 0.3693
Epoch 3/100
46/46 [==============================] - ETA: 0s - loss: 1.3302 - accuracy: 0.3533
Epoch 3: val_accuracy did not improve from 0.36932
46/46 [==============================] - 34s 728ms/step - loss: 1.3302 - accuracy: 0.3533 - val_loss: 1.3463 - val_accuracy: 0.3693
Epoch 4/100
46/46 [==============================] - ETA: 0s - loss: 1.3228 - accuracy: 0.3587
Epoch 4: val_accuracy did not improve from 0.36932
46/46 [==============================] - 33s 723ms/step - loss: 1.3228 - accuracy: 0.3587 - val_loss: 1.3450 - val_accuracy: 0.3693
Epoch 5/100
46/46 [===============